# E E 440 Final Project
*Submission deadline: December 9, 2019, 11:59PM*

**The final grading is based on:**
 - Functionality of your algorithm
 - GUI design•Rendering result
 - You need to include at least 3 images and the rendering results from your algorithm
 
**You will receive extra points if:**
 - Output is outstanding
 - Creative ideas
 - Interesting combinations or add-ons
 - Implementation is challenging
 
Please follow the homework submission guidelines. Be sure to zip all the necessary files including codes, images, and a project report. **In your project report, you should specify:**
 - Effects/functions you want to achieve
 - Details of the algorithm and implementation
 - Instruction on how to run all functionalities of your program
 - Comments on what you have learned and new features that can be added in the future.

In [1]:
from tkinter import filedialog
import tkinter as tk
import cv2
import PIL.Image, PIL.ImageTk
import numpy as np

In [113]:
# global variables
MARGIN = 10  # px
MAXDIM = 512
SLIDER_SIZE = 25
SV = 1  # show value of scale (1 or 0)

class App():
    def __init__(self, window, window_title, image_path="lena.bmp"):
        self.window = window
        self.window.title(window_title)
        
        # Load an image using OpenCV
        self.cv_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        # Get the image dimensions (OpenCV stores image data as NumPy ndarray)
        self.height, self.width, no_channels = self.cv_img.shape
        
        ''' Image Display Related Code'''
        # Create a FRAME that can fit the images
        self.frame1 = tk.Frame(self.window, width=self.width, height=self.height)
        self.frame1.pack(fill=tk.BOTH)        
        
        # Create a FRAME for original image
        self.frame_original = tk.Frame(self.frame1, width=self.width, height=self.height)
        self.frame_original.pack(side=tk.LEFT)
        
        # Create a CANVAS for original image
        self.canvas0 = tk.Canvas(self.frame_original, width=MAXDIM, height=MAXDIM+(3*MARGIN))
        self.canvas0.pack()
        
        # Create a FRAME for changing image
        self.frame_new = tk.Frame(self.frame1, width=self.width, height=self.height)
        self.frame_new.pack(side=tk.LEFT)
        
        # Create a CANVAS for changing image
        self.canvas1 = tk.Canvas(self.frame_new, width=MAXDIM, height=MAXDIM+(3*MARGIN))
        self.canvas1.pack()

        # Use PIL (Pillow) to convert the NumPy ndarray to a PhotoImage
        self.photoOG = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        self.photo = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
        
        # Add a PhotoImage to the Canvas (original)
        self.canvas0.create_image(MAXDIM//2, MAXDIM//2, image=self.photoOG)
        
        # Add a PhotoImage to the Canvas (changing effects)
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
        # Write labels for both images
        self.canvas0.create_text(
           MAXDIM//2, MAXDIM+(2*MARGIN), 
           font="Tahoma 16",text="Original Photo")
        self.canvas1.create_text(
           MAXDIM//2, MAXDIM+(2*MARGIN), 
           font="Tahoma 16",text="Changing Photo")
        
####################################################################################

        ''' Feature Display Code'''
        # Create a FRAME that can fit the features
        self.frame2 = tk.Frame(self.window, width=self.width, height=self.height//2)
        self.frame2.pack(expand=1, fill=tk.X)
        
        # GUI Decription Text
        self.label_og = tk.Label(self.frame2, text="How-to-use GUI", font="Tahoma 20 bold")
        self.label_og.pack(anchor=tk.W)
        self.label_og = tk.Label(
            self.frame2, font="Tahoma 16", justify=tk.LEFT,
            text="This is Jenny Cho's GUI for E E 440 final project.\nUse the scale on the right to change your image however you want! \nHave fun :-)")
        self.label_og.pack(anchor=tk.W)
        
        # Create a CANVAS for buttons
        self.canvas_but = tk.Canvas(self.frame2, bd=2, bg="floral white")
            ########################### ^^^ remove bg before turning it in ^^^
        self.canvas_but.pack(side=tk.LEFT, anchor=tk.W)
        
        # Create many empty CANVASes
        for i in range(2):
            self.canvas = tk.Canvas(self.frame2, bg="floral white")
                ########################### ^^^ remove bg before turning it in ^^^
            self.canvas.pack(side=tk.LEFT)
        
        # Create a CANVAS for additional features (1)
        #  blur & blemish & mosaic
        self.canvas_feat1 = tk.Canvas(self.frame2)
        self.canvas_feat1.pack(side=tk.LEFT, anchor=tk.E)
        
        # Create a CANVAS for additional features (2)
        #  brightness & contrast
        self.canvas_feat2 = tk.Canvas(self.frame2)
        self.canvas_feat2.pack(side=tk.LEFT, anchor=tk.E)
        
#         # Create a CANVAS for additional features (3)
#         #  
#         self.canvas_feat3 = tk.Canvas(self.frame2, bd=2)
#         self.canvas_feat3.pack(side=tk.LEFT, anchor=tk.E)
        
####################################################################################
    
        ''' Filter Feature Related Code -- ORDER MATTERS!!! '''
        # Create a BUTTON that loads image
        self.btn_load=tk.Button(
            self.canvas_but, text="Load", font="Tahoma 10", command=self.load)
        self.btn_load.config(height=3, width=9)
        self.btn_load.pack(anchor=tk.CENTER)
  
        # Create a BUTTON that resets the image
        self.btn_reset=tk.Button(
            self.canvas_but, text="Reset", font="Tahoma 8",
            command=self.reset, activeforeground="red3")
        self.btn_reset.config(height=1, width=9)
        self.btn_reset.pack(anchor=tk.CENTER)
        
        
        # Create a SCALE that lets the user blur the image
        self.scl_blur=tk.Scale(
            self.canvas_feat1, from_=1, to=50, orient=tk.HORIZONTAL, showvalue=SV,
            command = self.blur_image, sliderlength=SLIDER_SIZE, 
            label="Blur", font="Tahoma 12")
        self.scl_blur.pack(anchor=tk.SE)
        
        # Create a SCALE that lets the user remove blemishes
        self.scl_blmsh=tk.Scale(
            self.canvas_feat1, from_=0, to=10, orient=tk.HORIZONTAL, showvalue=SV,
            command = self.decBlemish_image, sliderlength=SLIDER_SIZE, 
            label="Blemish", font="Tahoma 12")
        self.scl_blmsh.pack(anchor=tk.SE)
        
        # Create a SCALE that lets the user mosaic the image
        self.scl_mosc=tk.Scale(
            self.canvas_feat1, from_=1, to=40, orient=tk.HORIZONTAL, showvalue=SV,
            command = self.mosaic_image, sliderlength=SLIDER_SIZE, 
            label="Mosaic", font="Tahoma 12")
        self.scl_mosc.pack(anchor=tk.SE)
        
        
        # Create a SCALE that lets the user change brightness
        self.scl_bright=tk.Scale(
            self.canvas_feat2, from_=-25, to=25, orient=tk.HORIZONTAL, showvalue=SV,
            command = self.brightness_image, sliderlength=SLIDER_SIZE, 
            label="Brightness", font="Tahoma 12")
        self.scl_bright.pack(anchor=tk.SE)
        
        # Create a SCALE that lets the user change contrast
        self.scl_contrast=tk.Scale(
            self.canvas_feat2, from_=-20, to=20, orient=tk.HORIZONTAL, showvalue=SV,
            command = self.contrast_image, sliderlength=SLIDER_SIZE, 
            label="Contrast", font="Tahoma 12")
        self.scl_contrast.pack(anchor=tk.SE)

        self.window.mainloop()

####################################################################################

    ''' Callback Functions'''
    # Callback for the "Blur" Scale
    def blur_image(self, k):
        k = self.scl_blur.get()
        self.NEWcv_img = cv2.blur(self.cv_img, (k, k))
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        

    # Callback for the "Blemish" Scale
    def decBlemish_image(self, k):
        k = self.scl_blmsh.get()
        # cancel out the effect
        if k == 0:
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.cv_img))
            self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        else:
            sigmaColor = 75
            sigmaSpace = 10
            self.NEWcv_img = cv2.bilateralFilter(self.cv_img, k*4, sigmaColor, sigmaSpace)
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img))
            self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
    
    
    # Brightness Effect
    def brightness_effect(self, k):
        print(k)
        img = cv2.cvtColor(self.cv_img, cv2.COLOR_BGR2HSV)
        img_v = (img[:,:,2].astype(np.uint16))+(k*5)  # add 5*k value channel
        print(img_v)
        img_v[img_v < 0] = 0      # adjust wrap around pixel (bottom)
        img_v[img_v > 255] = 255  # adjust overflow (top)
        img[:,:,2] = img_v.astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_HSV2BGR)
        return img
    
    # Callback for the "Brightness" Scale
    def brightness_image(self, k):
        k = self.scl_bright.get()
        self.NEWcv_img = self.brightness_effect(k)
        self.photo = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.NEWcv_img))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        
        
    # Callback for the "Contrast" Scale
    def contrast_image(self, k):
        k = self.scl_contrast.get()
        # gamma correction
        # https://www.pyimagesearch.com/2015/10/05/opencv-gamma-correction/
        # for k < 0, take the negative inverse, e.g. k=-2, it should be --> 1/2

        
    # Mosaic Effect
    def mosaic_effect(self, k):
        img_mosc = np.zeros_like(self.cv_img)
        for ch in range(3):  # all three bgr channels
            img = self.cv_img[:,:,ch]

            # save a "small photo" for every "k"
            img_small = img[0::k, 0::k]
            h, w = img_small.shape

            # new image frame
            x, y, ignore = self.cv_img.shape
            img_ch = np.zeros((x,y))

            # fill picture with mosaic-ed pixels
            for i in range(h):
                for j in range(w):
                    if ((i*k) < img.shape[0]) or ((j*k) < img.shape[1]):
                        img_ch[(i*k):(i*k)+k, (j*k):(j*k)+k] = img_small[i][j]

            img_mosc[:,:,ch] = img_ch
        return img_mosc
                
    # Callback for the "Mosaic" Scale
    def mosaic_image(self, k):
        k = self.scl_mosc.get()
        if k == 1:
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.cv_img))
            self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
        else:
            self.NEWcv_img = self.mosaic_effect(k)
            self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.NEWcv_img))
            self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)
    
    
    # Callback for the "Load" Button
    def load(self):
        image_path = filedialog.askopenfilename()
        if (image_path):
            self.cv_img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
            self.height, self.width, no_channels = self.cv_img.shape
            if (self.height > MAXDIM) or (self.width > MAXDIM):
                # resize image to have max dim be 512 
                maxval = max(self.height, self.width)
                new_H = int(self.height/maxval)*MAXDIM
                new_W = int(self.width/maxval)*MAXDIM
                self.cv_img = cv2.resize(self.cv_img, (new_H,new_W))
            self.reset()
            self.photoOG = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
            self.photo = PIL.ImageTk.PhotoImage(image=PIL.Image.fromarray(self.cv_img))
            self.canvas0.create_image(MAXDIM//2, MAXDIM//2, image=self.photoOG)
            self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo)

            
    # Callback for the "Reset" Button
    def reset(self):
        # Reset Scales
        self.scl_blur.set(1)      # blur
        self.scl_blmsh.set(0)     # blemish
        self.scl_bright.set(0)    # brightness
        self.scl_contrast.set(0)  # contrast
        self.scl_mosc.set(1)      # mosaic
        
        # Original Image
        self.photo = PIL.ImageTk.PhotoImage(image = PIL.Image.fromarray(self.cv_img))
        self.canvas1.create_image(MAXDIM//2, MAXDIM//2, image=self.photo, anchor=tk.CENTER)  

In [114]:
# Create a window and pass it to the Application object
# App(tk.Toplevel(), "Tkinter and OpenCV")
# App(tk.Tk(), "GUI Window", "jenny_resistor.bmp")
App(tk.Tk(), "GUI Window")

1
[[131 157 176 ... 167 168 169]
 [129 160 181 ... 168 169 169]
 [126 159 180 ... 168 169 170]
 ...
 [ 99 101 101 ...  35  31  28]
 [108 104 101 ...  36  31  27]
 [109 101  98 ...  37  31  27]]
6
[[156 182 201 ... 192 193 194]
 [154 185 206 ... 193 194 194]
 [151 184 205 ... 193 194 195]
 ...
 [124 126 126 ...  60  56  53]
 [133 129 126 ...  61  56  52]
 [134 126 123 ...  62  56  52]]
7
[[161 187 206 ... 197 198 199]
 [159 190 211 ... 198 199 199]
 [156 189 210 ... 198 199 200]
 ...
 [129 131 131 ...  65  61  58]
 [138 134 131 ...  66  61  57]
 [139 131 128 ...  67  61  57]]
8
[[166 192 211 ... 202 203 204]
 [164 195 216 ... 203 204 204]
 [161 194 215 ... 203 204 205]
 ...
 [134 136 136 ...  70  66  63]
 [143 139 136 ...  71  66  62]
 [144 136 133 ...  72  66  62]]
9
[[171 197 216 ... 207 208 209]
 [169 200 221 ... 208 209 209]
 [166 199 220 ... 208 209 210]
 ...
 [139 141 141 ...  75  71  68]
 [148 144 141 ...  76  71  67]
 [149 141 138 ...  77  71  67]]
10
[[176 202 221 ... 212 213 2

### Mosaic Effect Algorithm
Given an image and a paramter $k$ (user input from scale bar on GUI), a *mosaic* effect is applied to the image. $k$ is the number of pixels within a bigger pixel of the image. For example, if $k=3$, then each bigger pixel will be a $3*3$ "pixel" where the value is determined from sampling the original image, every 3 pixels in both x and y directions.

### Basic Display Algorithm
Each effect is an adjustable effect filter that is applied to the image. 
The ultimate goal is to build each effect on top of one another (e.g., blur a little AND mosaic).